<a href="https://colab.research.google.com/github/pfyeh1/ner_nyt/blob/main/eda_orgs_nytimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!ls "/content/drive/My Drive/_data/"

In [37]:
import os, sys
import pandas as pd
import numpy as np

import string
import re
from collections import Counter
from tqdm import tqdm

import pickle

#local_python_path = os.path.join(os.environ['HOME'], 'Documents', 'Python')
#os.chdir(local_python_path)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 800)

gpath = os.path.join('/content','drive', 'My Drive', '_data', 'nyt_articles')
os.chdir(gpath)

In [5]:
# infile
infile = "nyt_articles.pkl"

# infile path
infile_path = os.path.join(gpath, "output", infile)

# load dataframe from pkl
df = pd.read_pickle(infile_path)

In [6]:
df.columns

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section',
       'print_page', 'source', 'multimedia', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'type_of_material', '_id',
       'word_count', 'uri', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'subsection_name'],
      dtype='object')

In [7]:
cols = [
    'uri',
    'web_url',
    'snippet',
    'lead_paragraph',
    'keywords',
    'pub_date',
    'document_type',
    'news_desk',
    'section_name',
  ]

In [8]:
# don't need all the cols
df = df[cols].copy()

In [9]:
df.isnull().sum()

uri               0
web_url           0
snippet           0
lead_paragraph    0
keywords          0
pub_date          0
document_type     0
news_desk         0
section_name      0
dtype: int64

In [10]:
df['pub_date'] = pd.to_datetime(df['pub_date']).dt.strftime('%Y-%m-%d')

In [11]:
df.head()

,uri,web_url,snippet,lead_paragraph,keywords,pub_date,document_type,news_desk,section_name
0,nyt://article/df646720-840a-559b-9019-a38e073d374d,https://www.nytimes.com/2022/10/31/nyregion/rikers-inmate-stabbing-correction-officer.html,"The officer was seriously injured in the attack, which took place in the protective custody unit at Rikers, a union official said.","A city correction officer at the Rikers Island prison complex was seriously injured on Monday when a detainee stabbed him more than a dozen times in the back of the head, the correction officers’ union said in a statement.","[{'name': 'subject', 'value': 'Prison Guards and Corrections Officers', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Rikers Island Prison Complex', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Assaults', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Correction Officers Benevolent Assn', 'rank': 4, 'major': 'N'}]",2022-11-01,article,Metro,New York
1,nyt://article/bea5feb0-69e1-53a8-a0d0-eb8028756bfd,https://www.nytimes.com/2022/10/31/sports/baseball/world-series-astros-game-3-phillies.html,"The Houston Astros and the Philadelphia Phillies are tied at one game apiece. The series is scheduled to resume Tuesday, and each remaining game will be pushed back one day.","PHILADELPHIA — Game 3 of the World Series was postponed Monday because of rain, with the Houston Astros and the Philadelphia Phillies tied at one game apiece. The series is scheduled to resume Tuesday, and in place of eliminating the built-in travel day, each game will be pushed back one day.","[{'name': 'subject', 'value': 'Baseball', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'World Series', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Playoff Games', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Houston Astros', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Philadelphia Phillies', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'McCullers, Lance Jr (1993- )', 'rank': 6, 'major': 'N'}, {'name': 'persons', 'value': 'Syndergaard, Noah', 'rank': 7, 'major': 'N'}]",2022-11-01,article,Sports,Sports
2,nyt://article/b54435b9-a1e6-53a5-909a-58142f8ce981,https://www.nytimes.com/2022/10/31/crosswords/daily-puzzle-2022-11-01.html,Make some connections with Bruce Haight’s puzzle.,TUESDAY PUZZLE — Rumor has it that this is the 61st puzzle constructed by Bruce Haight for The Times. That’s a lot of puzzles!,"[{'name': 'subject', 'value': 'Crossword Puzzles', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Haight, Bruce', 'rank': 2, 'major': 'N'}]",2022-11-01,article,Games,Crosswords & Games
3,nyt://article/e860a43e-23b4-53d5-b1b7-9c87012e6e58,https://www.nytimes.com/2022/10/31/us/steven-pankey-murder-kidnapping-id.html,"The long-unsolved kidnapping and killing of Jonelle Matthews haunted the community of Greeley, about 50 miles north of Denver.","A Colorado jury on Monday convicted an Idaho man of murder and kidnapping in the 1984 disappearance and death of a 12-year-old girl who was killed days before Christmas, a case that went unsolved for decades and haunted the tight-knit community of Greeley, Colo., where she lived.","[{'name': 'subject', 'value': 'Murders, Attempted Murders and Homicides', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Kidnapping and Hostages', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Matthews, Jonelle', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Pankey, Steven D (1951- )', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Greeley (Colo)', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Idaho', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Decisions and Verdicts', 'rank': 7, 'major': 'N'}]",2022-11-01,article,Express,U.S.
4,nyt://video/bb3260bb-b2a2-5e3d-96be-4a9526b1667f,https://www.nytimes.com/video/us/politics/100000008610372/pelosi-attack-suspect-charges.html,"David DePape, who is accused of br

In [12]:
print(df['pub_date'].max())
print(df['pub_date'].min())

2023-10-22
2022-11-01


In [13]:
# filters
date = (df['pub_date']>='2023-10-07')
section = (df['section_name']=='World')
document_type = (df['document_type']=='article')

df1 = df.loc[date & section & document_type].copy()

In [14]:
# define the key-value pair you want to filter for
target_key = 'glocations'
target_value = 'Israel'

df2 = df1[df1['keywords'].apply(lambda x: any(entry['name'] == target_key and entry['value'] == target_value for entry in x))].copy()


In [17]:
print("No. of records before filter :", len(df1))
print("No. of records befafterore filter :", len(df2))

No. of records before filter : 301
No. of records befafterore filter : 126


In [18]:
# peek at first record of keywords field
df2.loc[df2['uri']=='nyt://article/ccd81853-4659-5a9d-b2e0-53489a78db39']['keywords']

44747    [{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Defense and Military Forces', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Palestinians', 'rank': 5, 'major': 'N'}, {'name': 'subject', 'value': 'Evacuations and Evacuees', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Refugees and Displaced Persons', 'rank': 7, 'major': 'N'}, {'name': 'subject', 'value': 'Terrorism', 'rank': 8, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 9, 'major': 'N'}, {'name': 'subject', 'value': 'Civilian Casualties', 'rank': 10, 'major': 'N'}]
Name: keywords, dtype: object

In [19]:
# turn dictionary in list into just list of vlaues
#df2['keywords_'] = df2['keywords'].apply(lambda x: [i['value'] for i in x])

In [20]:
import spacy.cli
spacy.cli.download("en_core_web_md")
import spacy
import en_core_web_md
#from spacy.pipeline import EntityRuler
nlp = spacy.load("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [21]:
ents = [
    'Hezbollah',
    'Hamas']

In [22]:
# initaliz empty list and add onewrd_nse_terrs to ent_patrns
ent_patrns = []

for ent in ents:
  ent_ = {"label":"ORG", "pattern":[{"LOWER":ent.lower()}]}
  ent_patrns.append(ent_)

In [23]:
ent_patrns

[{'label': 'ORG', 'pattern': [{'LOWER': 'hezbollah'}]},
 {'label': 'ORG', 'pattern': [{'LOWER': 'hamas'}]}]

In [24]:
# initialize EntityRuler
#ruler = EntityRuler(nlp, overwrite_ents = True, validate = True)

# initialize Entity Ruler
ruler = nlp.add_pipe("entity_ruler")

In [25]:
# add patterns to ruler
ruler.add_patterns(ent_patrns)

In [26]:
# some testing
doc = nlp("Hamas wants to attack aliens")

In [27]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Hamas ORG


In [28]:
# add ruler to pipeline
#nlp.add_pipe(ruler, before = "ner")

In [29]:
# function
def extract_named_ents(text):
  "use spacy to extract GPEs"
  ent_labels = ["GPE", "NORP", "ORG"]
  doc = nlp(text)
  items = [(x.text, x.label_) for x in doc.ents if x.label_ in ent_labels] # use to get NORP and GPE
  items_counter = Counter(items)#.most_common(10)
  return items_counter

In [30]:
# instaniate tqdm
tqdm.pandas()

In [38]:
# apply extraction function
df2['ents_extracted'] = df2['lead_paragraph'].str.lower().progress_apply(extract_named_ents)

100%|██████████| 126/126 [00:01<00:00, 86.23it/s]


In [43]:
df2.loc[df2['lead_paragraph'].str.contains(r'hezbollah', case = False, na = False)][['lead_paragraph','keywords', 'ents_extracted']].sample(n=2)

,lead_paragraph,keywords,ents_extracted
44366,"Shelling sounded out along Israel’s volatile northern border with Lebanon for a third consecutive day Tuesday, stoking fears on both sides of a repeat of 2006, when Israel fought a bloody monthlong war with Hezbollah, the Shiite group committed to the destruction of the Jewish state.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Hezbollah', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Lebanon', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Syria', 'rank': 5, 'major': 'N'}]","{('israel', 'GPE'): 2, ('lebanon', 'GPE'): 1, ('hezbollah', 'ORG'): 1, ('the shiite group committed', 'ORG'): 1, ('jewish', 'NORP'): 1}"
45557,"As Israel prepares a possible ground invasion of Gaza to destroy Hamas, the Palestinian militant group that controls the enclave, many are now turning to see what action Hezbollah, the armed group on Israel’s northern border, will take. For decades, Hezbollah has been a significant force in Lebanon, wielding both political power and military strength — all while engaging in tit-for-tat skirmishes with its southern enemy, Israel.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'War and Armed Conflicts', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Hezbollah', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Lebanon', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 6, 'major': 'N'}]","{('israel', 'GPE'): 3, ('gaza', 'GPE'): 1, ('hamas', 'GPE'): 1, ('palestinian', 'NORP'): 1, ('hezbollah', 'ORG'): 2, ('lebanon', 'GPE'): 1}"
